In [ ]:
"""To visualize which parts of the image are the most important for the each  
label for the classifier, let’s set up the Grad-CAM process."""
import tensorflow as tf
import cv2
#https://github.com/spirosdim/CNN_interpretability
#Primera versión en el TFM

def xray_CAM(model,img_array, last_conv_layer_name, pred_index): 
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


def superimpose(img_path, heatmap):
  # We use cv2 to load the original image
  img = cv2.imread(img_path)

  # We resize the heatmap to have the same size as the original image
  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

  # We convert the heatmap to RGB
  heatmap = np.uint8(255 * heatmap)

  # We apply the heatmap to the original image
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

  # 0.4 here is a heatmap intensity factor
  superimposed_img = heatmap * 0.4 + img

  ## Save the image to disk
  cv2.imwrite('result/xray_cam.jpg', superimposed_img)
  img_samp = cv2.imread('result/xray_cam.jpg')  #OpenCV has BGR order
  img_samp = cv2.cvtColor(img_samp, cv2.COLOR_BGR2RGB)  #matplotlib has RGB order
  
  return img_samp


In [ ]:
from PIL import Image
ef={}
write=False
effusion=[]
valores=[]

with open('resnet152/Consolidation.txt') as f:

    while True:
        line = f.readline()
        line=line.rstrip()
        if not line:
            break

        image, auc, size1, size2=line.split(":")

        if image not in effusion:
            image="nuevas_pruebas/train/" + image
            
            end=image.split("train_jpg/")[1]
            end=image.split("train_jpg/")[1]
            effusion.append("Consolidation/" + end)
            ef["Consolidation/" + end]=(float(auc), (int(size1), int(size2)))
            valores.append(float(auc))
        else:
            print(image)


In [ ]:
dict={}
dict["Atelectasis"]=ef

In [ ]:
len(dict["Atelectasis"])

In [ ]:
import math
def func_one(contours):
        primero=-1
        lista=[[]]
        for item in range(len(contours)):
            cnt = contours[item]
            x,y,w,h = cv2.boundingRect(cnt)
            area=w*h
            if area>primero:
                    lista[0]=[x,y,w,h]
                    primero=area
        if primero==-1:
            return []
        return lista

In [ ]:
remove=[]
for elem in dict["Atelectasis"].keys():
    s,size=dict["Atelectasis"][elem]
    if s<0.9:
        remove.append(elem)
    #if elem not in vect_auc[7]:
        #remove.append(elem)


for item in remove:
    dict["Atelectasis"].pop(item)
    


In [ ]:
len(dict["Atelectasis"])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
from PIL import Image
from scipy import ndimage
from skimage import exposure
import numpy as np

from skimage.filters import threshold_multiotsu

# Function that normalizes de image
def normalize_image(array):
	array = tf.cast(array,tf.float32) # Convert to float
	array = tf.math.multiply(array, 1./255) # Convert to range 0-1
	array = tf.math.subtract(array, 0.449) # Normalize((x-mean)/std)
	array = tf.math.multiply(array, 1./0.226)
	return array
n=0
lungs={}
for key_disease in dict.keys():
    print(key_disease)
    #l=len(dict[key_disease].keys())
    for ki,i in enumerate(dict[key_disease].keys()):
        #print(i)
            s,size=dict[key_disease][i]
            #plt.figure(figsize=(10, 10))
            #ax = plt.subplot(2, 1, 1)
            img_path= i
    
        
        

            grey_img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY)
            p2, p98 = np.percentile(grey_img, (0, 80))
            hist = exposure.rescale_intensity(grey_img, in_range=(p2, p98))
            #ret3,fondo = cv2.threshold(hist,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            ret3,fondo = cv2.threshold(hist,100,255,cv2.THRESH_BINARY)
            p=ndimage.binary_fill_holes(fondo).astype(int)

            #plt.axis('off')
            #plt.imshow(grey_img, cmap="gray")
            #plt.axis('off')
            #s='Grey img ' + str(s)
            #plt.title(s ,fontsize=15)
    

            #ax = plt.subplot(1, 9, 2)
            #plt.axis('off')
            contrast=hist
            #plt.imshow(contrast, cmap="gray")
            #plt.title('Contrast '  + str(ret3),fontsize=15)

            #ax = plt.subplot(1, 9, 3)
            #plt.axis('off')
            t0,t1=threshold_multiotsu(contrast, classes=3)
            print(t0,t1)
            
            ret3,th3 = cv2.threshold((contrast).astype("uint8"),t1,255,cv2.THRESH_BINARY)

            #ret3,th3 = cv2.threshold((contrast).astype("uint8"),205,255,cv2.THRESH_BINARY)
            #plt.imshow(th3)
            #plt.title('Contrast lungs ',fontsize=15)



            #ax = plt.subplot(1, 9, 4)
            #plt.axis('off')
            th3=ndimage.binary_erosion(th3, structure=np.ones((5,1)))
            th3fh=ndimage.binary_fill_holes(th3, structure=np.ones((1,3)))
            #plt.imshow(th3fh)
            #plt.title('Fill holes contrast',fontsize=15)
    


            #ax = plt.subplot(1, 9, 5)
            xor=np.logical_xor(th3,th3fh)
            xor=ndimage.binary_erosion(xor, structure=np.ones((1,3)))
            #ax = plt.subplot(2, 1, 2)
            #plt.axis('off')
            #plt.imshow(xor)
            #plt.title('Xor',fontsize=15)


            derecha=np.zeros(xor.shape)
            w,h=xor.shape
            derecha[:,0:h//2]=1
            izquierda=(derecha-1)*-1


            '''ax = plt.subplot(1, 9, 6)
            plt.imshow(grey_img, cmap="gray")'''
            xor_derecha=xor*derecha
            contours,hierarchy = cv2.findContours(xor_derecha.astype("uint8"), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

            lista=func_one(contours)
            colors=['r']
            app=[]
            for i,elem in enumerate(lista):
                #print(colors[i], elem)
                [x,y,w,h]=elem
                '''rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=colors[i], facecolor=(0.1,0,0,0.1))
                ax.add_patch(rect)
                

                ax.plot(x, y, 'o', color='b')  # top-left corner
                ax.plot(x+w, y+h, '*', color='c' ) # bottom-right corner'''
                app.append(elem)
            
            xor_izquierda=xor*izquierda
            contours,hierarchy = cv2.findContours(xor_izquierda.astype("uint8"), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

            lista=func_one(contours)
            colors=['b']
            for i,elem in enumerate(lista):
                #print(colors[i], elem)
                [x,y,w,h]=elem
                '''rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=colors[i], facecolor=(0.1,0,0,0.1))
                ax.add_patch(rect)
                

                ax.plot(x, y, 'o', color='b')  # top-left corner
                ax.plot(x+w, y+h, '*', color='c' ) # bottom-right corner'''
                app.append(elem)

            [xd,yd,wd,hd]=app[0]
            [xi,yi,wi,hi]=app[1]

            if yd<yi:
                app[1]=[xi,yd,wi,hi + (yi-yd)]
                '''rect = patches.Rectangle((xi, yd), wi, hi + (yi-yd), linewidth=2, edgecolor="black", facecolor=(0.1,0,0,0.1))
                ax.add_patch(rect)
                ax.plot(xi, yd, 'o', color='b')  # top-left corner
                ax.plot(xi+wi, (yd + hi + (yi-yd)), '*', color='c' ) # bottom-right corner'''
            else:
                app[0]=[xd,yi,wd,hd + (yd-yi)]
                '''rect = patches.Rectangle((xd, yi), wd, hd + (yd-yi), linewidth=2, edgecolor="black", facecolor=(0.1,0,0,0.1))
                ax.add_patch(rect)
                ax.plot(xd, yi, 'o', color='b')  # top-left corner
                ax.plot(xd+wd, (yd +hd + (yd-yi)), '*', color='c' ) # bottom-right corner'''
                

            lungs[img_path]=app
            [xd,yd,wd,hd]=app[0]
            [xi,yi,wi,hi]=app[1]

            fi=yi+hi
            fd=yd+hd


            if fd>fi:
                    app[1]=[xi,yi,wi,hi + (fd-fi)]
                    #rect = patches.Rectangle((xi, yi), wi,hi + (fd-fi), linewidth=2, edgecolor="pink", facecolor=(0.1,0,0,0.1))
                    #ax.add_patch(rect)

            else:
                    app[0]=[xd,yd,wd,hd + (fi-fd)]
                    #rect = patches.Rectangle((xd, yd), wd,hd+ (fi-fd), linewidth=2, edgecolor="purple", facecolor=(0.1,0,0,0.1))
                    #ax.add_patch(rect)

            lungs[img_path]=app       
        

                

            '''plt.tight_layout()
            plt.show()'''
   

In [ ]:
# IoU for object detection
def get_IoU(truth_coords, pred_coords):
    pred_area = pred_coords[2]*pred_coords[3]
    truth_area = truth_coords[2]*truth_coords[3]
    # coords of intersection rectangle
    x1 = max(truth_coords[0], pred_coords[0])
    y1 = max(truth_coords[1], pred_coords[1])
    x2 = min(truth_coords[2], pred_coords[2])
    y2 = min(truth_coords[3], pred_coords[3])
    # area of intersection rectangle
    interArea = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    # area of prediction and truth rectangles
    boxTruthArea = (truth_coords[2] - truth_coords[0] + 1) * (truth_coords[3] - truth_coords[1] + 1)
    boxPredArea = (pred_coords[2] - pred_coords[0] + 1) * (pred_coords[3] - pred_coords[1] + 1)
    # intersection over union 
    if boxTruthArea + boxPredArea!=0.0:
        new_iou = 2*interArea / float(boxTruthArea + boxPredArea)  #Coeficiente de Sørensen-Dice , Dice Coefficient (F1 Score)
    else:
        new_iou=0
    if boxTruthArea + boxPredArea - interArea !=0.0:
        iou=interArea /float(boxTruthArea + boxPredArea - interArea)
    else:
        iou=0.0
    if boxTruthArea!=0 and boxPredArea!=0:
        iou_min=interArea /min(float(boxTruthArea),float(boxPredArea))
    else:
        iou_min=0
    purity=interArea/boxPredArea
    if boxTruthArea!=0:
        integrity=interArea/boxTruthArea
    else:
        integrity=0
    #print(purity, integrity)
    if purity!=0.0 and integrity!=0.0:
        iou_decop=1/((1/purity)+(1/integrity)-1)
    else:
        iou_decop=0.0
    alphas=[0.2,0.5,0.7]
    propuestas=[]
    if purity!=0.0 and integrity!=0.0:
        for alpha in alphas:
            propuesta=1/(((1-alpha)/purity)+((1+alpha)/integrity)-1)
            propuestas.append(propuesta)
    else:
        propuestas=[0.0,0.0,0.0]
    if purity!=0.0 and integrity!=0.0:
        beta=4
        fbeta=(1+beta**2)*((purity*integrity)/((beta**2*purity)+integrity))
    else:
        fbeta=0.0

    return new_iou,iou, iou_min, purity, integrity, fbeta, [x1, y1, x2-x1, y2-y1]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
from PIL import Image
from scipy import ndimage


# Function that normalizes de image
def normalize_image(array):
	array = tf.cast(array,tf.float32) # Convert to float
	array = tf.math.multiply(array, 1./255) # Convert to range 0-1
	array = tf.math.subtract(array, 0.449) # Normalize((x-mean)/std)
	array = tf.math.multiply(array, 1./0.226)
	return array

model=tf.keras.models.load_model("resnet152/xb_resenet152_pooling_recall_0.81_precision_0.81_auc_0.90.h5")

j=0
new_heatmaps_grad_cam=[]
old_heatmaps_grad_cam=[]
heatmaps_guided_grad_cam=[]
heatmaps_grad_cam_plus=[]
heatmaps_score_cam=[]
heatmaps_faster_score_cam=[]
heatmaps=[]
n=0
heatmaps_dict={}
for key_disease in dict.keys():
    print(key_disease)
    for ki,img_path in enumerate(dict[key_disease].keys()):

            s,size=dict[key_disease][img_path]
            img_path= img_path
            _img=Image.open(img_path) #.convert("RGB")
            _img = normalize_image(_img)
            _img = np.expand_dims(_img, axis=0)
            img = cv2.imread(img_path)

            grey_img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(grey_img,(5,5),0)
            ret3,th3 = cv2.threshold(blur,50,255,cv2.THRESH_BINARY)
            p=ndimage.binary_fill_holes(th3).astype(int)


          

            #,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung 
            #Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
            
            #heatmap = xray_CAM(model,_img, 'conv5_block16_2_conv', 5)
            heatmap = xray_CAM(model,_img, 'conv5_block3_3_conv', 5)
            heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
            ap=heatmap #*p
            heatmaps.append(ap)
            heatmaps_dict[img_path]=heatmap
   


            '''impose=superimpose(img_path, heatmap)
            impose[:,:,0]=impose[:,:,0]*p
            impose[:,:,1]=impose[:,:,1]*p
            impose[:,:,2]=impose[:,:,2]*p'''


            '''plt.figure(figsize=(20, 20))
            ax = plt.subplot(1, 7, 3)
            plt.axis('off')
            plt.imshow(impose)
            #plt.title('Model ' + str(s),fontsize=15)

            plt.tight_layout()
            plt.show()
            print(n)'''
            '''n+=1
            if n==5:
                break'''
          
    



In [ ]:
import cv2
from cv2 import imshow as cv2_imshow
import matplotlib.pyplot as plt
import os
import numpy as np

class BBoxerwGradCAMarea(): #Contorno más grande 
    
    def __init__(self,heatmap,image_path,bbox_scale_list, size,t1):
        self.heatmap = heatmap
        self.image_path = image_path
        self.scale_list = bbox_scale_list
        self.size=size
        self.t1=t1
        self.og_img, self.smooth_heatmap = self.heatmap_smoothing() 
        if self.size!=-1:
                self.lista_bbox_coords, self.lista_poly_coords, self.grey_img, self.contours = self.form_bboxes()
        else:
                self.bbox_coords, self.poly_coords, self.grey_img, self.contours = self.form_bboxes()

        
    def heatmap_smoothing(self):
        og_img = cv2.imread(self.image_path)
        '''
        The minimum pixel value will be mapped to the minimum output value (alpha - 0)
        The maximum pixel value will be mapped to the maximum output value (beta - 155)
        Linear scaling is applied to everything in between.
        These values were chosen with trial and error using COLORMAP_JET to deliver the best pixel saturation for forming contours.
        '''
        heatmapshow = cv2.normalize(self.heatmap, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        heatmapshow=cv2.cvtColor(heatmapshow, cv2.COLOR_GRAY2RGB)
        #heatmapshow = cv2.applyColorMap(heatmapshow, cv2.COLORMAP_JET)
        #heatmapshow=cv2.cvtColor(heatmapshow, cv2.COLOR_RGB2BGR)
        
        return og_img, heatmapshow
    
    def get_steps(self):
        return self.heatmap, self.smooth_heatmap
    
    def show_smoothheatmap(self):
        plt.imshow(self.smooth_heatmap)
        plt.axis("off")
        plt.show()
    
    def show_grey_img(self):
        grey_img = cv2.cvtColor(self.smooth_heatmap, cv2.COLOR_BGR2GRAY)
        plt.imshow(grey_img, cmap="gray")
        plt.axis("off")
        plt.show()
        
    def show_bboxrectangle(self):
        #print(self.bbox_coords)
        cv2.rectangle(self.og_img,
                      (self.bbox_coords[0],self.bbox_coords[1]),
                      (self.bbox_coords[0]+self.bbox_coords[2],self.bbox_coords[1]+self.bbox_coords[3]),
                      (255,0,0),5)
        plt.imshow(self.og_img)
        plt.axis("off")
        plt.show()
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
    
    def show_contouredheatmap(self):
        img_col = cv2.merge([self.grey_img,self.grey_img,self.grey_img]) # merge channels to create color image (3 channels)
        cv2.fillPoly(img_col, self.contours, [36,255,12]) # fill contours on 3 channel image
        plt.imshow(img_col)
        plt.axis("off")
        plt.show()
        
    def show_bboxpolygon(self):
        cv2.polylines(self.og_img,self.poly_coords,True,(0,0,0),2)
        plt.imshow(self.og_img)
        plt.axis("off")
        plt.show()
    
    def form_bboxes(self):
        if self.t1==256:
            grey_img = cv2.cvtColor(self.smooth_heatmap, cv2.COLOR_BGR2GRAY)
            #blur = cv2.GaussianBlur(grey_img,(5,5),0)
            ret,thresh  = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            contours,hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            
        
        else:
            grey_img = cv2.cvtColor(self.smooth_heatmap, cv2.COLOR_BGR2GRAY)
            ret,thresh = cv2.threshold(grey_img,self.t1,255,cv2.THRESH_BINARY)
            contours,hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        

        #print(ret)
        #plt.imshow(thresh)
        #plt.show()
        not_box=True
        mas_grande=False
        lista=False
        anterior=0
        lista_poly_coords=[]
        lista_bbox_coords=[]
        points=[]


        if self.size==-1:
            mas_grande=True

        if self.size==1:
            valores=[]
            lista=True


        for item in range(len(contours)):
            cnt = contours[item]
            if mas_grande:   #contorno más grande  Self.size:-1
                x,y,w,h = cv2.boundingRect(cnt)
                points.append(cnt)
                area=w*h
                if area>anterior:
                    poly_coords = [cnt] # polygon coordinates are based on contours
                    x = int(x*self.scale_list[0]) # rescaling the boundary box based on user input
                    y = int(y*self.scale_list[1])
                    w = int(w*self.scale_list[2])
                    h = int(h*self.scale_list[3])
                    not_box=False
                    anterior=area

            if lista:  #lista de todos los contornos
                #if len(cnt)>10:
                x,y,w,h = cv2.boundingRect(cnt) # x, y is the top left corner, and w, h are the width and height respectively
                lista_poly_coords.append(cnt) # polygon coordinates are based on contours
                x = int(x*self.scale_list[0]) # rescaling the boundary box based on user input
                y = int(y*self.scale_list[1])
                w = int(w*self.scale_list[2])
                h = int(h*self.scale_list[3])
                lista_bbox_coords.append([x,y,w,h])
                not_box=False
                    

            
        if not_box:
            return [], [],[],[],[]
        if mas_grande:
            return [x,y,w,h], poly_coords, grey_img, contours
        else:
            return lista_bbox_coords, lista_poly_coords, grey_img, contours
 
            
                
    def get_bboxes(self):
        if self.size==-1:
            return self.bbox_coords 
        else:
            return self.lista_bbox_coords

       

In [ ]:
len(heatmaps)

In [ ]:
new_df=pd.read_csv('train.csv')
len(new_df["image_id"].unique())

In [ ]:
import cv2
from cv2 import imshow as cv2_imshow
import matplotlib.pyplot as plt
import os
import numpy as np

class BBoxerwGradCAMAX1():
    
    def __init__(self,heatmap,image_path,bbox_scale_list,t1):
        self.heatmap = heatmap
        self.image_path = image_path
        self.scale_list = bbox_scale_list
        self.t1=t1
        self.og_img, self.smooth_heatmap = self.heatmap_smoothing() 
        self.bbox_coords, self.poly_coords, self.grey_img, self.contours = self.form_bboxes()
        

        
    def heatmap_smoothing(self):
        og_img = cv2.imread(self.image_path)
        '''
        The minimum pixel value will be mapped to the minimum output value (alpha - 0)
        The maximum pixel value will be mapped to the maximum output value (beta - 155)
        Linear scaling is applied to everything in between.
        These values were chosen with trial and error using COLORMAP_JET to deliver the best pixel saturation for forming contours.
        '''
        '''heatmapshow = cv2.normalize(self.heatmap, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        heatmapshow=cv2.cvtColor(heatmapshow, cv2.COLOR_GRAY2RGB)
        heatmapshow = cv2.applyColorMap(heatmapshow, cv2.COLORMAP_JET)
        heatmapshow=cv2.cvtColor(heatmapshow, cv2.COLOR_RGB2BGR)'''

        heatmapshow = cv2.normalize(heatmap, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        heatmapshow=cv2.cvtColor(heatmapshow, cv2.COLOR_GRAY2RGB)
        
        return og_img, heatmapshow
    
    def show_smoothheatmap(self):
        plt.imshow(self.smooth_heatmap)
        plt.show()
        
    def show_bboxrectangle(self):
        #print(self.bbox_coords)
        cv2.rectangle(self.og_img,
                      (self.bbox_coords[0],self.bbox_coords[1]),
                      (self.bbox_coords[0]+self.bbox_coords[2],self.bbox_coords[1]+self.bbox_coords[3]),
                      (255,255,0),2)
        plt.imshow(self.og_img)
        plt.show()
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
    
    def show_contouredheatmap(self):
        img_col = cv2.merge([self.grey_img,self.grey_img,self.grey_img]) # merge channels to create color image (3 channels)
        cv2.fillPoly(img_col, self.contours, [36,255,12]) # fill contours on 3 channel image
        plt.imshow(img_col)
        plt.show()
        
    def show_bboxpolygon(self):
        cv2.polylines(self.og_img,self.poly_coords,True,(0,0,0),2)
        plt.imshow(self.og_img)
        plt.show()
    
    def form_bboxes(self):
        grey_img = cv2.cvtColor(self.smooth_heatmap, cv2.COLOR_BGR2GRAY)
        kmax=np.max(grey_img)
        ret,thresh = cv2.threshold(grey_img,kmax-self.t1,255,cv2.THRESH_BINARY)
        contours,hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        not_box=True
        poly_coords=[]
        bbox_coords=[]

        anterior=0
        for item in range(len(contours)):
                cnt = contours[item]
                if len(cnt)>anterior:
                    x,y,w,h = cv2.boundingRect(cnt) # x, y is the top left corner, and w, h are the width and height respectively
                    poly_coords = [cnt] # polygon coordinates are based on contours
                    x = int(x*self.scale_list[0]) # rescaling the boundary box based on user input
                    y = int(y*self.scale_list[1])
                    w = int(w*self.scale_list[2])
                    h = int(h*self.scale_list[3])
                    bbox_coords=[x,y,w,h]
                    not_box=False
                    anterior=len(cnt)      
   
                
        if not_box:
            return [], [],[],[]
        else:
            return bbox_coords, poly_coords, grey_img, contours
 
    def get_list(self):
        return self.bbox_coords, self.poly_coords, self.grey_img, self.contours
                
    def get_bboxes(self):
         return self.bbox_coords

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import skimage as skimage
from skimage.filters import threshold_multiotsu
bbox_scaling = [1,1,1,1]
maximo_dict={}
j=0
brillante_dict={}
print("BBOX MAX")
for key_disease in dict.keys():
    print(key_disease)
    l=len(dict[key_disease].keys())
    for ki,i in enumerate(dict[key_disease].keys()):
        #print(i)
            s,size=dict[key_disease][i]
            img = cv2.imread(i)
            heatmap = cv2.resize(heatmaps[ki], (img.shape[1], img.shape[0])) 

            img =np.array(img, np.float32) / 255  
            #fig=plt.figure(figsize=(50, 50))

            imgs=i.split("Consolidation/")
            imgs=imgs[1].split(".jpg")
            a=new_df[new_df['image_id']==imgs[0]]
            k_index=a[a['class_name']=='Consolidation']


            '''ax = plt.subplot(11, 11, 1)
            plt.imshow(img)
            plt.axis("off")
            #ax.set_title(i.split("train_jpg/")[1])
            ax.set_title("Original image", fontsize=20)'''

            x1=lungs[i][0][0] +lungs[i][0][2]
            x2=lungs[i][1][0]
            th=(lungs[i][0][2] + lungs[i][1][2])/3
            #print(x2-x1)
            heatmap[:, 160]=0
           


            #ax = plt.subplot(11, 11, 2)
            impose=superimpose(i, heatmap)
            #plt.axis("off")
            #plt.imshow(impose)
            #ax.set_title("Heatmap", fontsize=20)

            bbox = BBoxerwGradCAMAX1(heatmap,  i, bbox_scaling, 50)
            lista=bbox.get_bboxes()
            brillante_dict[imgs[0]]=lista
            maximo=[[0.0,[], 0],[0.0,[], 0]]

            h = cv2.normalize(heatmap, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            h=cv2.cvtColor(h, cv2.COLOR_GRAY2RGB)
            h = cv2.cvtColor(h, cv2.COLOR_BGR2GRAY)
            #ret3,fondo = cv2.threshold(h,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            #print(threshold_multiotsu(h, classes=3))
            if len(h[h>0])!=0:
                t0,t1, t2=threshold_multiotsu(h, classes=4)
                #print(t0,t1,t2)
            else:
                t0=0
                t1=0
                #print("else")
        
            for ki,k in enumerate((range(t0,t2,1))):
            #for ki,k in enumerate(reversed(range(220,250,150))):
                #print("Nivel de gris: " + str(k))
                nivel_gris=k
                #print("Nivel de gris: ", k)
                #bbox = BBoxerwGradCAM(heatmap,  i, bbox_scaling, -1, k)
                bbox = BBoxerwGradCAMarea(heatmap,  i, bbox_scaling, 1, k)

                lista=bbox.get_bboxes()
                # Create figure and axes
                # ax = plt.subplot(11, 11, ki+3)
                #ax.set_title("Minimum brightness level: " +str(k), fontsize=20)
                #ax.set_title("Bounding box at: " +str(nivel_gris), fontsize=20)


                # Display the image
                #plt.imshow(impose)
                #plt.axis("off")

                

                '''colors=['r', 'b']
                for ki,elem in enumerate(lungs[i]):#[:2]:
                    [x,y,w,h]=elem
                    #print([x,y,w,h])
                    rect1=patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=colors[ki], facecolor='none')
                    ax.add_patch(rect1)
                    ax.plot(x, y, 'o', color='b')  # top-left corner
                    ax.plot(x+w, y+h, '*', color='c' ) # bottom-right corner'''
   
                
           
                for ki,elem in enumerate(lista):#[:2]:
                    [x,y,w,h]=elem
                    '''rect1=patches.Rectangle((x, y), w-1, h-1, linewidth=3, edgecolor="green", facecolor='none')
                    ax.add_patch(rect1)
                    ax.plot(x, y, 'o', color='b')  # top-left corner
                    ax.plot(x+w-1, y+h-1, '*', color='c' ) # bottom-right corner'''
                    xl,yl,wl,hl=lungs[i][0]
                    #xl,yl,wl,hl=xl,yl+2*hl/3,wl-wl/3,2*hl/3
                    new_iou,iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xl,yl,xl+wl,yl+hl], [x, y, x+w, y+h])
                    area=w*h
                    if new_iou>maximo[0][0]: # and 300<area<26000:
                        maximo[0]=[new_iou,propuestas, area]

                    else:
            
                        xl,yl,wl,hl=lungs[i][1]
                        #xl,yl,wl,hl=xl,yl+2*hl/3,2*wl-wl/3,2*hl/3
                        new_iou,iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xl,yl,xl+wl,yl+hl], [x, y, x+w, y+h])
                        #print(iou)
                        area=w*h
                        if new_iou>maximo[1][0]: # and 300<area<26000 : #and iou>iou_derecha:
                            maximo[1]=[new_iou,propuestas, area]


                if nivel_gris>=t2-10:
                    if maximo[0][0]==0.0 and maximo[1][0]==0.0:
                        for ki,elem in enumerate(lista):#[:2]:
                            [x,y,w,h]=elem
                            xl,yl,wl,hl=lungs[i][0]
                            if x<(xl+wl):
                                maximo[0]=[1,[x, y, w, h], False]
                            else:
                                maximo[1]=[1,[x, y, w, h], False]


                if maximo[0][0]!=0.0 and maximo[1][0]!=0.0 or nivel_gris>=t2-10:
                    colors=['purple', 'pink']
                    for ki,elem in enumerate(maximo):#[:2]:
                        #print(str(colors[ki]) +" "+ str(elem[0]))
                        if elem[1]!=[]:
                            num,[x,y,w,h], solap=elem
                            rect1=patches.Rectangle((x, y), w-1, h-1, linewidth=3, edgecolor=colors[ki], facecolor='none')
                            #ax.add_patch(rect1)
                            #ax.plot(x, y, 'o', color='b')  # top-left corner
                            #ax.plot(x+w-1, y+h-1, '*', color='c' ) # bottom-right corner
                      
                
                    maximo_dict[imgs[0]]=maximo
                    break
            j+=1

            
    

In [ ]:
new_df["b"],new_df["num_rad"]=new_df['rad_id'].str.split("R", 1).str
new_df.drop(columns=["b"], inplace=True)

In [ ]:
new_df["num_rad"] = pd.to_numeric(new_df["num_rad"])
new_df.sort_values(inplace=True, by=['num_rad'])
new_df

In [ ]:
effusion=new_df[new_df["class_name"]=="Consolidation"]
print(effusion["num_rad"].unique())

In [ ]:
# IoU for object detection
def suma(vector):
    # coords of intersection rectangle
    if len(vector)==2:
        x1 = min(vector[0][0], vector[1][0])
        y1 = min(vector[0][1], vector[1][1])
        x2 = max(vector[0][2], vector[1][2])
        y2 = max(vector[0][3], vector[1][3])
    elif len(vector)==3:
        x1 = min(vector[0][0], vector[1][0], vector[2][0])
        y1 = min(vector[0][1], vector[1][1], vector[2][1])
        x2 = max(vector[0][2], vector[1][2], vector[2][2])
        y2 = max(vector[0][3], vector[1][3], vector[2][3])
    else:
        x1 = min(vector[0][0], vector[1][0], vector[2][0], vector[3][0])
        y1 = min(vector[0][1], vector[1][1], vector[2][1], vector[3][1])
        x2 = max(vector[0][2], vector[1][2], vector[2][2], vector[3][2])
        y2 = max(vector[0][3], vector[1][3], vector[2][3], vector[3][3])
    
    return [x1,y1,x2,y2]

In [ ]:
# IoU for object detection
def suma_rad(vector):
        if len(vector)>=1:
            x1=np.min([vector[i][0] for i in range(len(vector))])
            y1=np.min([vector[i][1] for i in range(len(vector))])
            x2=np.max([vector[i][2] for i in range(len(vector))])
            y2=np.max([vector[i][3] for i in range(len(vector))])      
            return [x1,y1,x2,y2, vector[0][4]]
        else:
            return vector

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
bbox_scaling = [1,1,1,1]
suma_dict={}
derecha_izquierda_dict={}
dict_auc={}
vector_rads=[8,9,10]
incluir=True

for key_disease in dict.keys():
    print(key_disease)
    l=len(dict[key_disease].keys())
    for ki,i in enumerate(dict[key_disease].keys()):
            
            #print(i)
            s,size=dict[key_disease][i]
            dict_auc[i]=s
       
            img = cv2.imread(i)
            heatmap = cv2.resize(heatmaps[ki], (img.shape[1], img.shape[0])) 

            img =np.array(img, np.float32) / 255  
            #fig=plt.figure(figsize=(5, 5))

            imgs=i.split("Consolidation/")
            imgs=imgs[1].split(".jpg")
            a=new_df[new_df['image_id']==imgs[0]]
            k_index=a[a['class_name']=='Consolidation']

            #impose=superimpose(i, heatmap)


            #ax = plt.subplot(1, 1, 1)
            #ax.set_title(i + " " + str(s))


            # Display the image
            #plt.imshow(impose)
            #colors=["red", "blue"]

            '''for ki,elem in enumerate(lungs[i]):#[:2]:
                    [x,y,w,h]=elem
                    #print([x,y,w,h])
                    rect1=patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=colors[ki], facecolor='none')
                    ax.add_patch(rect1)
                    ax.plot(x, y, 'o', color='b')  # top-left corner
                    ax.plot(x+w, y+h, '*', color='c' ) # bottom-right corner
                #print(get_IoU([x_min,y_min,x_max,y_max], [x, y, x+w, y+h]))'''
            
            colors=["pink", "orange", "navy", "snow", "grey", "green"]
            descartados=[]
            derecha=[]
            izquierda=[]
            for ki,k in enumerate(k_index.index):      
                x_min = new_df['x_min'][k]*(320/size[1])
                y_min = new_df['y_min'][k]*(320/size[0])
                x_max = new_df['x_max'][k]*(320/size[1])
                y_max = new_df['y_max'][k]*(320/size[0])
                #respecto a los pulmones (radiografía)
                #rect1=patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=2, edgecolor=colors[ki], facecolor='none')
                #ax.add_patch(rect1)
                #ax.plot(x, y, 'o', color='b')  # top-left corner
                num_rad=new_df['num_rad'][k]
                if num_rad not in vector_rads:
                   incluir=False
                   break

                
                xl,yl,wl,hl=lungs[i][0]
                xl,yl,wl,hl=xl,yl,wl,2*hl
                new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xl,yl,xl+wl,yl+hl],[x_min, y_min, x_max, y_max])
            
                color="black"
                if iou>0.0:
                    derecha.append([x_min, y_min, x_max, y_max, num_rad])
                    color="pink"
                
                xl,yl,wl,hl=lungs[i][1]
                xl,yl,wl,hl=xl,yl,wl,2*hl
                new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xl,yl,xl+wl,yl+hl],[x_min, y_min, x_max, y_max])
                #print(iou)
                if iou>0.0:
                    izquierda.append([x_min, y_min, x_max, y_max, num_rad])
                    color="orange"

                if color=="black":
                    descartados.append([x_min, y_min, x_max, y_max, num_rad])
                
                

                #rect = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=2, edgecolor=color, facecolor=(0.0,0.0,0.0,0.0))
                #ax.add_patch(rect)
            if incluir:    
                parar=False
                color="black"
                #print(descartados)
                for descartado in descartados:
                    xd,yd,wd,hd, num_rad=descartado
                    for der in derecha:
                        xder,yder,wder,hder, num_rad_der=der
                        new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([ xder,yder,wder,hder],[xd,yd,wd,hd])
                        if iou>0.0:
                            derecha.append([xd,yd,wd,hd, num_rad])
                            parar=True
                            color="pink"
                            break
                    if not parar:
                        for izq in izquierda:
                            xizq,yizq,wizq,hizq, num_rad_izq=izq
                            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xizq,yizq,wizq,hizq],[xd,yd,wd,hd])
                            if iou>0.0:
                                izquierda.append([xd,yd,wd,hd,num_rad])
                                parar=True
                                color="orange"
                                break
                    #if parar:
                        #rect = patches.Rectangle((xd, yd), wd-xd, hd-yd, linewidth=2, edgecolor=color, facecolor=(0.0,0.0,0.0,0.0))
                        #ax.add_patch(rect)

                derecha_izquierda_dict[imgs[0]]=[derecha,izquierda]
                bbox_derecha=False
                bbox_izquierda=False
                if len(derecha)>1:
                    xd, yd, wd, hd=suma(derecha)  
                    #rect = patches.Rectangle((xd, yd), wd-xd, hd-yd, linewidth=2, facecolor=(0.0,0.0,0.0,0.5))
                    #ax.add_patch(rect)
                    bbox_derecha=True
                

                if len(izquierda)>1:
                    xi, yi, wi, hi=suma(izquierda)  
                    #rect = patches.Rectangle((xi, yi), wi-xi, hi-yi, linewidth=2, facecolor=(0.0,0.0,0.0,0.5))
                    #ax.add_patch(rect)
                    bbox_izquierda=True
                
                if bbox_derecha and bbox_izquierda:
                    suma_dict[i]=[[ xd, yd, wd, hd], [ xi, yi, wi, hi]]
                elif bbox_derecha:
                    suma_dict[i]=[[ xd, yd, wd, hd],[]]
                elif bbox_izquierda:
                    suma_dict[i]=[[], [xi, yi, wi, hi]]

            else:
                incluir=True
            
            
            #plt.show()
            #break

In [ ]:
print(len(heatmaps))
print(len(derecha_izquierda_dict))

In [ ]:
len(maximo_dict)

In [ ]:
nuevo_derecha_izquierda_dict={}
for key in derecha_izquierda_dict.keys():
    nuevo_derecha_izquierda_dict[key]=[[],[]]
    if derecha_izquierda_dict[key][0]!=[]:
        ocho=[]
        nueve=[]
        diez=[]
        for elem in derecha_izquierda_dict[key][0]:
            if elem[4]==8:
                ocho.append(elem)
            if elem[4]==9:
                nueve.append(elem)
            if elem[4]==10:
                diez.append(elem)
        if suma_rad(ocho)!=[]:
            nuevo_derecha_izquierda_dict[key][0].append(suma_rad(ocho))
        if suma_rad(nueve)!=[]:
            nuevo_derecha_izquierda_dict[key][0].append(suma_rad(nueve))
        if suma_rad(diez)!=[]:
            nuevo_derecha_izquierda_dict[key][0].append(suma_rad(diez))
    if derecha_izquierda_dict[key][1]!=[]:
        ocho=[]
        nueve=[]
        diez=[]
        for elem in derecha_izquierda_dict[key][1]:
            if elem[4]==8:
                ocho.append(elem)
            if elem[4]==9:
                nueve.append(elem)
            if elem[4]==10:
                diez.append(elem)
        if suma_rad(ocho)!=[]:
            nuevo_derecha_izquierda_dict[key][1].append(suma_rad(ocho))
        if suma_rad(nueve)!=[]:
            nuevo_derecha_izquierda_dict[key][1].append(suma_rad(nueve))
        if suma_rad(diez)!=[]:
            nuevo_derecha_izquierda_dict[key][1].append(suma_rad(diez))
    
    




In [ ]:
print(len(nuevo_derecha_izquierda_dict))

In [ ]:
print(len(nuevo_derecha_izquierda_dict))

In [ ]:
import seaborn as snNew
import pandas as pdNew
import matplotlib.pyplot as pltNew
from scipy.spatial.distance import directed_hausdorff


#correspondencia={8:0, 9:1,10:2, 11:0, 12:1,14:2, 15:0, 16:1}
correspondencia={8:0, 9:1,10:2}
total_new_iou_der=[]
total_iou_der=[]
total_min_der=[]
total_hd_der=[]
total_purity_der=[]
total_integrity_der=[]
total_fbeta=[]
total_propuesta_1_der=[]
total_propuesta_2_der=[]
total_propuesta_3_der=[]
total_new_iou_izq=[]
total_iou_izq=[]
total_min_izq=[]
total_hd_izq=[]
total_purity_izq=[]
total_integrity_izq=[]
total_iou_decop_izq=[]
total_propuesta_1_izq=[]
total_propuesta_2_izq=[]
total_propuesta_3_izq=[]
area=[]

num=5
for j,key in enumerate(nuevo_derecha_izquierda_dict.keys()):
    array_new_iou_der=np.zeros((num,num))
    np.fill_diagonal(array_new_iou_der, 1)
    array_iou_der=np.zeros((num,num))
    np.fill_diagonal(array_iou_der, 1)
    array_min_iou_der=np.zeros((num,num))
    np.fill_diagonal(array_min_iou_der, 1)
    array_hd_der=np.zeros((num,num))
    array_purity_der=np.zeros((num,num))
    np.fill_diagonal(array_purity_der, 1)
    array_integrity_der=np.zeros((num,num))
    np.fill_diagonal(array_integrity_der, 1)
    array_fbeta=np.zeros((num,num))
    np.fill_diagonal(array_fbeta, 1)

    


    
    fig=plt.figure(figsize=(5, 5))
    numm, t, valores=maximo_dict[key][0]
    axr = plt.subplot(1, 1, 1)
    plt.axis('off')
    i="Consolidation/" + key + ".jpg"
    #print(key+ " - " + str(dict_auc[i]))

    img = cv2.imread(i)
    heatmap = cv2.resize(heatmaps_dict[i], (img.shape[1], img.shape[0])) 
    impose=superimpose(i, heatmap)
    plt.imshow(impose)
    
    fig=plt.figure(figsize=(5, 5))
    axr_todo = plt.subplot(1, 1, 1)
    plt.axis('off')


    impose=superimpose(i, heatmap)
    plt.imshow(impose)
    #fig=plt.figure(figsize=(20, 20))
    color={8:"green", 9:"red", 10:"blue", 11: "yellow", 12: "pink", 13:"white", 14:"green", 15:"grey", 16:"purple"}
    if t!=[] and nuevo_derecha_izquierda_dict[key][0]!=[]:
        for pivote in nuevo_derecha_izquierda_dict[key][0]:
            xp,yp,wp,hp,num_piv=pivote
            area.append((wp-xp)*(hp-yp))
            for elem in nuevo_derecha_izquierda_dict[key][0]:
                    x,y,w,h, num_elem=elem
                    new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xp,yp,wp,hp], [x, y, w, h])
                    array_new_iou_der[correspondencia[num_piv]][correspondencia[num_elem]]=new_iou
                    array_iou_der[correspondencia[num_piv]][correspondencia[num_elem]]=iou
                    array_min_iou_der[correspondencia[num_piv]][correspondencia[num_elem]]=iou_min
                    array_purity_der[correspondencia[num_piv]][correspondencia[num_elem]]=purity
                    array_integrity_der[correspondencia[num_piv]][correspondencia[num_elem]]=integrity
                    array_fbeta[correspondencia[num_piv]][correspondencia[num_elem]]=iou_decop
                   

                    rect1=patches.Rectangle((x, y), w-x, h-y, linewidth=3, edgecolor=color[num_elem], facecolor='none')
                    axr.add_patch(rect1)
                    #axr.plot(x, y, 'o', color='b')  # top-left corner
                    #axr.plot(w, h, '*', color='c' ) # bottom-right corner
                    rect1=patches.Rectangle((x, y), w-x, h-y, linewidth=3, edgecolor=color[num_elem], facecolor='none')
                    axr_todo.add_patch(rect1)
                    #axr_todo.plot(x, y, 'o', color='b')  # top-left corner
                    #axr_todo.plot(w, h, '*', color='c' ) # bottom-right corner
                                  
                

            numm,[xm,ym,wm,hm], valores=maximo_dict[key][0]
            rect1=patches.Rectangle((xm, ym), wm, hm, linewidth=3, edgecolor="purple", facecolor='none', linestyle="--")
            axr_todo.add_patch(rect1)
            #axr_todo.plot(xm, ym, 'o', color='b')  # top-left corner
            #axr_todo.plot(xm+wm, ym+hm, '*', color='c' ) # bottom-right corner
            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xp,yp,wp,hp], [xm, ym, xm+wm, ym+hm])
            array_new_iou_der[correspondencia[num_piv]][-2]=new_iou
            array_iou_der[correspondencia[num_piv]][-2]=iou
            array_min_iou_der[correspondencia[num_piv]][-2]=iou_min
            array_purity_der[correspondencia[num_piv]][-2]=purity
            array_integrity_der[correspondencia[num_piv]][-2]=integrity
            array_fbeta[correspondencia[num_piv]][-2]=iou_decop


         
            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xm, ym, xm+wm, ym+hm], [xp,yp,wp,hp])
            array_new_iou_der[-2][correspondencia[num_piv]]=new_iou
            array_iou_der[-2][correspondencia[num_piv]]=iou
            array_min_iou_der[-2][correspondencia[num_piv]]=iou_min
            
            array_purity_der[-2][correspondencia[num_piv]]=purity
            array_integrity_der[-2][correspondencia[num_piv]]=integrity
            array_fbeta[-2][correspondencia[num_piv]]=iou_decop


            
            [xb,yb,wb,hb]=brillante_dict[key]
            rect1=patches.Rectangle((xb, yb), wb, hb, linewidth=3, edgecolor="black", facecolor='none', linestyle="--")
            axr.add_patch(rect1)
            #axr.plot(xb, yb, 'o', color='b')  # top-left corner
            #axr.plot(xb+wb, yb+hb, '*', color='c' ) # bottom-right corner
            rect1=patches.Rectangle((xb, yb), wb, hb, linewidth=3, edgecolor="black", facecolor='none', linestyle="--")
            axr_todo.add_patch(rect1)
            #axr_todo.plot(xb, yb, 'o', color='b')  # top-left corner
            #axr_todo.plot(xb+wb, yb+hb, '*', color='c' ) # bottom-right corner
            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xp,yp,wp,hp], [xb, yb, xb+wb, yb+hb])
            array_new_iou_der[correspondencia[num_piv]][-1]=new_iou
            array_iou_der[correspondencia[num_piv]][-1]=iou
            array_min_iou_der[correspondencia[num_piv]][-1]=iou_min
            array_purity_der[correspondencia[num_piv]][-1]=purity
            array_integrity_der[correspondencia[num_piv]][-1]=integrity
            array_fbeta[correspondencia[num_piv]][-1]=iou_decop


            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU( [xb, yb, xb+wb, yb+hb],[xp,yp,wp,hp])

            array_new_iou_der[-1][correspondencia[num_piv]]=new_iou
            array_iou_der[-1][correspondencia[num_piv]]=iou
            array_min_iou_der[-1][correspondencia[num_piv]]=iou_min
            
            array_purity_der[-1][correspondencia[num_piv]]=purity
            array_integrity_der[-1][correspondencia[num_piv]]=integrity
            array_fbeta[-1][correspondencia[num_piv]]=iou_decop #'''

           
            
        
        new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xm,ym,xm+wm,ym+hm], [xb, yb, xb+wb, yb+hb])
        array_new_iou_der[-2][-1]=new_iou
        array_iou_der[-2][-1]=iou
        array_min_iou_der[-2][-1]=iou_min
        array_purity_der[-2][-1]=purity
        array_integrity_der[-2][-1]=integrity
        array_fbeta[-2][-1]=iou_decop


        new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xb, yb, xb+wb, yb+hb], [xm,ym,xm+wm,ym+hm])

        array_new_iou_der[-1][-2]=new_iou
        array_iou_der[-1][-2]=iou
        array_min_iou_der[-1][-2]=iou_min
            
        array_purity_der[-1][-2]=purity
        array_integrity_der[-1][-2]=integrity
        array_fbeta[-1][-2]=iou_decop #'''
        

     
        total_iou_der.append(array_iou_der)
        total_new_iou_der.append(array_new_iou_der)
        total_min_der.append(array_min_iou_der)
        total_purity_der.append(array_purity_der)
        total_integrity_der.append(array_integrity_der)
        total_fbeta.append(array_fbeta)
        


        '''DetaFrame_cm = pdNew.DataFrame(array_new_iou_der, ["R8", "R9", "R10", "RDM", "NAIF"], ["R8", "R9", "R10", "RDM", "NAIF"])
        ax = plt.subplot(3, 3, 1)
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('New IOU der')'''

        '''ax = plt.subplot(3, 3, 1)
        DetaFrame_cm = pdNew.DataFrame(array_iou_der,["R8", "R9", "R10", "RDM",  "N"], ["R8", "R9", "R10", "RDM", "N"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Normal IOU der')'''
    
        '''ax = plt.subplot(3, 3, 3)
        DetaFrame_cm = pdNew.DataFrame(array_min_iou_der, ["R8", "R9", "R10", "RDM", "NAIF"], ["R8", "R9", "R10", "RDM", "NAIF"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Min IOU der')'''

        '''ax = plt.subplot(3, 3, 2)
        DetaFrame_cm = pdNew.DataFrame(array_purity_der,["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
        snNew.heatmap(DetaFrame_cm, annot=True,  fmt=".1f", ax=ax, cmap=snNew.cubehelix_palette(as_cmap=True))
        ax.set_title('Purity der')'''

        '''ax = plt.subplot(3, 3, 3)
        DetaFrame_cm = pdNew.DataFrame(array_integrity_der,["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Integrity der')'''

        '''ax = plt.subplot(3, 3, 6)
        DetaFrame_cm = pdNew.DataFrame(array_fbeta,["R8", "R9", "R10", "RDM", "NAIF"], ["R8", "R9", "R10", "RDM", "NAIF"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Fbeta-Score der')'''
       




   
    array_new_iou_izq=np.zeros((num,num))
    np.fill_diagonal(array_new_iou_izq, 1)
    array_iou_izq=np.zeros((num,num))
    np.fill_diagonal(array_iou_izq, 1)
    array_min_iou_izq=np.zeros((num,num))
    np.fill_diagonal(array_min_iou_izq, 1)
    array_hd_izq=np.zeros((num,num))
    array_purity_izq=np.zeros((num,num))
    np.fill_diagonal(array_purity_izq, 1)
    array_integrity_izq=np.zeros((num,num))
    np.fill_diagonal(array_integrity_izq, 1)
    array_iou_decop_izq=np.zeros((num,num))
    np.fill_diagonal(array_iou_decop_izq, 1)
    array_propuesta_1_izq=np.zeros((num,num))
    np.fill_diagonal(array_propuesta_1_izq, 1)
    array_propuesta_2_izq=np.zeros((num,num))
    np.fill_diagonal(array_propuesta_2_izq, 1)
    array_propuesta_3_izq=np.zeros((num,num))
    np.fill_diagonal(array_propuesta_3_izq, 1)

    #np.fill_diagonal(array_min_iou_izq, 1)
    

    numm,t,valores=maximo_dict[key][1]
    if t!=[] and nuevo_derecha_izquierda_dict[key][1]!=[]:
    
        #fig=plt.figure(figsize=(10, 10))
        for pivote in nuevo_derecha_izquierda_dict[key][1]:
            xp,yp,wp,hp,num_piv=pivote
            area.append((wp-xp)*(hp-yp))
            for elem in nuevo_derecha_izquierda_dict[key][1]:
                    x,y,w,h, num_elem=elem
                    new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xp,yp,wp,hp], [x, y, w, h])
                    array_new_iou_izq[correspondencia[num_piv]][correspondencia[num_elem]]=new_iou
                    array_iou_izq[correspondencia[num_piv]][correspondencia[num_elem]]=iou
                    array_min_iou_izq[correspondencia[num_piv]][correspondencia[num_elem]]=iou_min
                    array_purity_izq[correspondencia[num_piv]][correspondencia[num_elem]]=purity
                    array_integrity_izq[correspondencia[num_piv]][correspondencia[num_elem]]=integrity
                    array_iou_decop_izq[correspondencia[num_piv]][correspondencia[num_elem]]=iou_decop
                    rect1=patches.Rectangle((x, y), w-x, h-y, linewidth=3, edgecolor=color[num_elem], facecolor='none')
                    axr.add_patch(rect1)
                    #axr.plot(x, y, 'o', color='b')  # top-left corner
                    #axr.plot(w, h, '*', color='c' ) # bottom-right corner
                    rect1=patches.Rectangle((x, y), w-x, h-y, linewidth=3, edgecolor=color[num_elem], facecolor='none')
                    axr_todo.add_patch(rect1)
                    #axr_todo.plot(x, y, 'o', color='b')  # top-left corner
                    #axr_todo.plot(w, h, '*', color='c' ) # bottom-right corner




            numm,[xm,ym,wm,hm], valores=maximo_dict[key][1]
            rect1=patches.Rectangle((xm, ym), wm, hm, linewidth=3, edgecolor="purple", facecolor='none', linestyle="--")
            axr_todo.add_patch(rect1)
            #axr_todo.plot(xm, ym, 'o', color='b')  # top-left corner
            #axr_todo.plot(xm+wm, ym+hm, '*', color='c' ) # bottom-right corner
            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xp,yp,wp,hp], [xm, ym, xm+wm, ym+hm])
            array_new_iou_izq[correspondencia[num_piv]][-2]=new_iou
            array_iou_izq[correspondencia[num_piv]][-2]=iou
            array_min_iou_izq[correspondencia[num_piv]][-2]=iou_min
            array_purity_izq[correspondencia[num_piv]][-2]=purity
            array_integrity_izq[correspondencia[num_piv]][-2]=integrity
            array_iou_decop_izq[correspondencia[num_piv]][-2]=iou_decop

           


            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xm, ym, xm+wm, ym+hm],[xp,yp,wp,hp])
            array_new_iou_izq[-2][correspondencia[num_piv]]=new_iou
            array_iou_izq[-2][correspondencia[num_piv]]=iou
            array_min_iou_izq[-2][correspondencia[num_piv]]=iou_min
            array_purity_izq[-2][correspondencia[num_piv]]=purity
            array_integrity_izq[-2][correspondencia[num_piv]]=integrity
            array_iou_decop_izq[-2][correspondencia[num_piv]]=iou_decop





            [xb,yb,wb,hb]=brillante_dict[key]
            rect1=patches.Rectangle((xb, yb), wb, hb, linewidth=3, edgecolor="black", facecolor='none', linestyle="--")
            axr.add_patch(rect1)
            #axr.plot(xb, yb, 'o', color='b')  # top-left corner
            #axr.plot(xb+wb, yb+hb, '*', color='c' ) # bottom-right corner
            rect1=patches.Rectangle((xb, yb), wb, hb, linewidth=3, edgecolor="black", facecolor='none', linestyle="--")
            axr_todo.add_patch(rect1)
            #axr_todo.plot(xb, yb, 'o', color='b')  # top-left corner
            #axr_todo.plot(xb+wb, yb+hb, '*', color='c' ) # bottom-right corner
            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xp,yp,wp,hp], [xb, yb, xb+wb, yb+hb])
            array_new_iou_der[correspondencia[num_piv]][-1]=new_iou
            array_iou_der[correspondencia[num_piv]][-1]=iou
            array_min_iou_der[correspondencia[num_piv]][-1]=iou_min
            array_purity_der[correspondencia[num_piv]][-1]=purity
            array_integrity_der[correspondencia[num_piv]][-1]=integrity
            array_fbeta[correspondencia[num_piv]][-1]=iou_decop


            new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU( [xb, yb, xb+wb, yb+hb], [xp,yp,wp,hp])

            array_new_iou_der[-1][correspondencia[num_piv]]=new_iou
            array_iou_der[-1][correspondencia[num_piv]]=iou
            array_min_iou_der[-1][correspondencia[num_piv]]=iou_min
            
            array_purity_der[-1][correspondencia[num_piv]]=purity
            array_integrity_der[-1][correspondencia[num_piv]]=integrity
            array_fbeta[-1][correspondencia[num_piv]]=iou_decop


        new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xm,ym,xm+wm,ym+hm], [xb, yb, xb+wb, yb+hb])
        array_new_iou_der[-2][-1]=new_iou
        array_iou_der[-2][-1]=iou
        array_min_iou_der[-2][-1]=iou_min
        array_purity_der[-2][-1]=purity
        array_integrity_der[-2][-1]=integrity
        array_fbeta[-2][-1]=iou_decop


        new_iou, iou, iou_min, purity, integrity, iou_decop, propuestas=get_IoU([xb, yb, xb+wb, yb+hb],[xm,ym,xm+wm,ym+hm])

        array_new_iou_der[-1][-2]=new_iou
        array_iou_der[-1][-2]=iou
        array_min_iou_der[-1][-2]=iou_min
            
        array_purity_der[-1][-2]=purity
        array_integrity_der[-1][-2]=integrity
        array_fbeta[-1][-2]=iou_decop #'''

        
        total_iou_izq.append(array_iou_izq)
        total_new_iou_izq.append(array_new_iou_izq)
        total_min_izq.append(array_min_iou_izq)
        total_hd_izq.append(array_hd_izq)
        total_purity_izq.append(array_purity_izq)
        total_integrity_izq.append(array_integrity_izq)
        total_iou_decop_izq.append(array_iou_decop_izq)



              
        '''DetaFrame_cm = pdNew.DataFrame(array_new_iou_izq,["R8", "R9", "R10", "RDM", "NAIF"], ["R8", "R9", "R10", "RDM", "NAIF"] )
        ax = plt.subplot(3, 3, 1)
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('New IOU izq')'''

        '''ax = plt.subplot(3, 3, 4)
        DetaFrame_cm = pdNew.DataFrame(array_iou_izq, ["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Normal IOU izq')'''
    
        '''ax = plt.subplot(3, 3, 3)
        DetaFrame_cm = pdNew.DataFrame(array_min_iou_izq, ["R8", "R9", "R10", "RDM", "NAIF"], ["R8", "R9", "R10", "RDM", "NAIF"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Min IOU izq')'''


        '''ax = plt.subplot(3, 3, 5)
        DetaFrame_cm = pdNew.DataFrame(array_purity_izq, ["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Purity izq')'''


        '''ax = plt.subplot(3, 3, 6)
        DetaFrame_cm = pdNew.DataFrame(array_integrity_izq, ["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
        snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
        ax.set_title('Integrity izq')'''

        '''ax = plt.subplot(3, 3, 6)
        DetaFrame_cm = pdNew.DataFrame(array_iou_decop_izq, ["R8", "R9", "R10", "RDM", "NAIF"], ["R8", "R9", "R10", "RDM", "NAIF"])
        snNew.heatmap(DetaFrame_cm, annot=True,  ax=ax)
        ax.set_title('FBeta-Score izq')'''
        #break
    #plt.axis("false")
    plt.show()

        
       
     
 
    

    
    

In [ ]:
fig=plt.figure(figsize=(12,5))
snNew.set(font_scale=1.5)

'''purity=total_purity_der+total_purity_izq
DetaFrame_cm = pdNew.DataFrame(np.round(np.sum(purity, axis=0)/len(purity),2), ["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
ax = plt.subplot(1, 3, 1)
snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
ax.set_title('Purity', fontsize=20)'''

integrity=total_integrity_der+total_integrity_izq
ax = plt.subplot(1, 2, 1)
DetaFrame_cm = pdNew.DataFrame(np.round(np.sum(integrity, axis=0)/len(integrity),2),  ["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
ax.set_title('Integrity', fontsize=20)

iou=total_new_iou_der+total_new_iou_izq
ax = plt.subplot(1, 2, 2)
DetaFrame_cm = pdNew.DataFrame(np.round(np.sum(iou, axis=0)/len(iou),2),  ["R8", "R9", "R10", "RDM", "N"], ["R8", "R9", "R10", "RDM", "N"])
snNew.heatmap(DetaFrame_cm, annot=True, ax=ax)
ax.set_title('DSC', fontsize=20)
    





plt.tight_layout()
plt.show()

